# ★ Feedback ★
## - RAPIDS 모듈을 colab에서 사용할 수 있습니다.
## - 현석님 올려주신 코드와 거의 비슷한데, tfidf model 구현 시 parameter가 다릅니다. (stop_words=None, binary=True, max_features=25000)
## - KNN 대신 COSINE SIMILARITY DISTANCE 라는 개념을 사용했는데, 왜 그런건지 이유를 아직 발견 못했습니다.
## - 미세한 parameter들 변경하면서 test 해볼 예정입니다.(max_features, cts[k,]>0.7 등)

# COLAB에 RAPIDS 설치하기

In [1]:
COLAB = True
default_dir = None

if COLAB:
    # from google.colab import drive
    # drive.mount('/content/drive')
    # default_dir = '/content'

    # Install RAPIDS
    !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
    !bash rapidsai-csp-utils/colab/rapids-colab.sh

    import sys, os

    dist_package_index = sys.path.index('/usr/local/lib/python3.7/dist-packages')
    # ValueError: '/usr/local/lib/python3.6/dist-packages' is not in list
    sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.7/site-packages'] + sys.path[dist_package_index:]
    sys.path
    exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

    # intall miniconda
    !wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
    !chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
    !bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

    # install RAPIDS packages
    !conda install -q -y --prefix /usr/local -c conda-forge \
    -c rapidsai-nightly/label/cuda10.0 -c nvidia/label/cuda10.0 \
    cudf cuml

    # set environment vars
    import shutil
    sys.path.append('/usr/local/lib/python3.6/site-packages/')
    os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
    os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

    # copy .so files to current working dir
    # FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/lib/libcudf.so'
    for fn in ['libcudf.so', 'librmm.so']:
        shutil.copy('/usr/local/lib/'+fn, os.getcwd())

    # ModuleNotFoundError: No module named 'conda'
    !conda install -c nvidia nvstrings==0.1.0
    !conda install -c rapidsai -c numba -c conda-forge -c defaults cudf=0.4.0

    # intall miniconda
    !wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
    !chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
    !bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

    !conda install -c pytorch faiss-gpu cuda92

    # install RAPIDS packages
    !conda install -q -y --prefix /usr/local -c conda-foFileNotFoundError: [Errno 2] No such file or directory: '/usr/local/lib/libcudf.so'rge \
    -c rapidsai-nightly/label/cuda10.0 -c nvidia/label/cuda10.0 \
    cudf cuml
    # FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/lib/libcudf.so'
    # https://github.com/rapidsai/cuml/issues/25
    # Running conda install -c pytorch faiss-gpu cuda92 prior to the cuML install resolves the issue.

    # set environment vars
    import sys, os, shutil
    sys.path.append('/usr/local/lib/python3.6/site-packages/')
    os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
    os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

    # copy .so files to current working dir
    # FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/lib/libcudf.so'
    for fn in ['libcudf.so', 'librmm.so']:
        shutil.copy('/usr/local/lib/'+fn, os.getcwd())

else: default_dir = '../input/shopee-product-matching'

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 213, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 213 (delta 22), reused 3 (delta 0), pack-reused 171
Receiving objects: 100% (213/213), 64.29 KiB | 10.71 MiB/s, done.
Resolving deltas: 100% (84/84), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT SCRIPT CHANGES: Colab has updated to Python 3.7, and now runs our STABLE and NIGHTLY versions (0.18 and 0.19)!  PLEASE update your older install script code as follows:
	!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.18

	import sys, os

	dist_package_index = sys.path.index('/usr/local/lib/python3.7/dist-packages')
	sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.7/site-packages'] + sys.path[dist_package_index:]
	sys.path
	exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), global

FileNotFoundError: ignored

## - 위 코드 실행하면 RAPIDS 버전 입력하라고 나올텐데 '0.18' 입력하고 enter

## - 약 20분정도 시간 소요되며 중간에 에러뜨면서 멈출텐데 cudf, cuml 사용하는 데는 이상 없음 (적어도 아래 코드에서는)

# RAM 관리

In [5]:
LIMIT = 1
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), 'Physical GPUs', len(logical_gpus), 'Logical GPUs')
    except RuntimeError as e:
        print(e)
    print('Restrict TensorFlow to max %iGB GPU RAM'%LIMIT)
    print('so RAPIDS can use %iGB GPU RAM'%(16-LIMIT))
else:
    print('Non Accelerator detected')

1 Physical GPUs 1 Logical GPUs
Restrict TensorFlow to max 1GB GPU RAM
so RAPIDS can use 15GB GPU RAM


# 라이브러리

In [12]:
import tensorflow as tf
import pandas as pd
import numpy as np
from os.path import join
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB3, EfficientNetB5, EfficientNetB6
import gc
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer

# 공통 변수 모음

In [3]:
COSINE_SIMILARITY = True   # ==> COSINE SIMILARITY DISTANCE 사용할 지 여부

Text_KNN_Follow_Up = False # ==> KNN 사용할 지 여부

# 함수 모음

In [9]:
def combine_for_match(row):
    x = np.concatenate([row.text_pred, row.phash_pred])
    return ' '.join(np.unique(x))

def combine_for_cv(row):
    x = np.concatenate([row.text_pred, row.phash_pred])
    return np.unique(x)

def get_metric(col):
    def f1_score(row):
        # np.intersect1d(x, y) : 두 개의 배열 x, y 의 교집합을 정렬하여 반환
        n= len(np.intersect1d(row['target'], row[col]))
        return 2*n / (len(row['target']) + len(row[col]))
    return f1_score

# Train DataSet (cudf 활용)

In [6]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle_data/shopee-product-matching/train.csv')
cu_df = cudf.DataFrame(df)
cu_df.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


# 예측값과 비교해서 사용할 Data group 정리

In [8]:
# 'label_group' data를 기준으로 posting_id 값 재정렬
# 즉, 'target' data는 우리의 label(정답)과 같은 역할
tmp = df.groupby('label_group').posting_id.agg('unique').to_dict()
df['target'] = df.label_group.map(tmp)

# 'image_phash' data를 기준으로 posting_id 값 재정렬
# 완전 동일한 이미지만 같은 label로 정렬하는 것과 같은 의미
tmp = df.groupby('image_phash').posting_id.agg('unique').to_dict()
df['phash_pred'] = df.image_phash.map(tmp)

# TFIDF 벡터화 (cuml의 TfidfVectorizer 사용)

In [7]:
# tfidf vectorizer model 만들기 (parameter 비교해주세요.)
model = TfidfVectorizer(stop_words=None, binary=True, max_features=25000)

# text_embedding 값 구하기
text_embeddings = model.fit_transform(cu_df.title).toarray()

print('text embeddings shape',text_embeddings.shape)

text embeddings shape (34250, 25000)


# text_embedding 값의 predict 결과 구하기

In [13]:
preds = []
CHUNK = 1024*4

print('비슷한 제목 찾기 시작')

CTS = len(df) // CHUNK
if len(df) % CHUNK != 0: CTS += 1

for i in range(CTS):

    a = i * CHUNK
    b = (i+1) * CHUNK
    b = min(b, len(df))
    print('chunk', a, 'to', b)

    if COSINE_SIMILARITY :
        # COSINE SIMILARITY DISTANCE 사용할 경우
        cts = cupy.matmul(text_embeddings, text_embeddings[a:b].T).T

        for k in range(b-a):
            idx = cupy.where(cts[k,]>0.7)[0]
            o = df.iloc[cupy.asnumpy(idx)].posting_id.values
            preds.append(o)
    else:
        # KNN 사용할 경우
        distances, indices = model.kneighbors(text_embeddings[a:b,])

        for k in range(b-a):
            idx = cupy.where(indices[k,]<6.0)[0]
            o = df.iloc[cupy.asnumpy(idx)].posting_id.values
            preds.append(o)

gc.collect()

df['text_pred'] = preds

비슷한 제목 찾기 시작
chunk 0 to 4096
chunk 4096 to 8192
chunk 8192 to 12288
chunk 12288 to 16384
chunk 16384 to 20480
chunk 20480 to 24576
chunk 24576 to 28672
chunk 28672 to 32768
chunk 32768 to 34250


# text_pred 값 & phash_pred 값 섞어서 최종 prediction 값 정렬 후 f1 score 구하기

In [14]:
df['mix_pred'] = df.apply(combine_for_cv, axis=1)

df['f1'] = df.apply(get_metric('mix_pred'), axis=1)

print('CV score :', df.f1.mean())

CV score : 0.6732140696292076


# text 값과 image_phash 값으로만 구한 최종 match 값

In [15]:
df['matches'] = df.apply(combine_for_match, axis=1)
df.head()

,posting_id,image,image_phash,title,label_group,target,phash_pred,text_pred,mix_pred,f1,matches
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]",[train_129225211],"[train_129225211, train_2278313361]","[train_129225211, train_2278313361]",1.000000,train_129225211 train_2278313361
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,"[train_3386243561, train_3423213080]",[train_3386243561],[train_3386243561],[train_3386243561],0.666667,train_3386243561
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,"[train_2288590299, train_3803689425]",[train_2288590299],[train_2288590299],[train_2288590299],0.666667,train_2288590299
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,"[train_2406599165, train_3342059966]",[train_2406599165],"[train_2406599165, train_3576714541, train_150...","[train_1508100548, train_1744956981, train_240...",0.285714,train_1508100548 train_1744956981 train_240659...
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,"[train_3369186413, train_921438619]",[train_3369186413],[train_3369186413],[train_3369186413],0.666667,train_3369186413
